# Grading an assignment on Canvas with the API

These tools were primarily developed to support my instruction of the University of British Columbia's Master of Data Science program.

In [1]:
import json
import os
from datetime import datetime, timedelta
from canvasapi import Canvas
import requests
import re
from tqdm import tqdm

In [4]:
API_URL = "https://canvas.ubc.ca/" # default is canvas.ubc
API_KEY = os.getenv("CANVAS_API")  # canvas.ubc instructor token
COURSE_CODE = 53659
canvas = Canvas(API_URL, API_KEY)
course = canvas.get_course(COURSE_CODE)

- Find the id of the assignment we want to grade:

In [41]:
[(_.name, _.id) for _ in course.get_assignments()]

[('lab1', 664385), ('lab2', 680571)]

- Use the desired id to retrieve the assignment and save it as `assignment`

In [6]:
assignment = course.get_assignment(664385)

- Okay, now we want to extract the autograde from every assignment, and upload it to Canvas.
- In the assignments, I've added an "autograde cell" which runs all tests and prints out "Autograde points: n / total"
- So I'm downloading every submission, using regex to extract that `n`, and then uploading
- Each submission for an assignment has a method `.edit()`, we can pass in a dictionary of the form: `{rubric_id: {"points": n}}`, where `rubric_id` is the id of the rubric element/row you want to update (you can look at all the id's with `assignment.rubric`.
- I'm catching any students who didn't run the cell or didn't submit
- As always, interacting with the canvas api is a little yucky here, but once you get it working it's a time-saver!

In [ ]:
no_submission = []
user_dict = {_.id: _.name for _ in course.get_users()}

for _ in assignment.rubric:
    if "Autograded Exercises" in _.values():
        autograde_rubric_id = assignment.rubric[-1]['id']

for ass in tqdm([_ for _ in assignment.get_submissions()]):
    crit_grade_comments = {}
    if ass.submission_type:
        with requests.session() as s:
            html = s.get(ass.attachments[0]['url']).text
            try:
                score = int(re.findall(r'Autograde points: (\d) / \d', html)[0])
            except:
                no_submission.append(user_dict[ass.user_id])
            crit_grade_comments[autograde_rubric_id] = {"points": str(score)}
            ass.edit(rubric_assessment=crit_grade_comments)
    else:
        no_submission.append(user_dict[ass.user_id])

 15%|█▍        | 20/136 [00:42<04:18,  2.23s/it]